# Experiment with vecsim

In [1]:
# Import vecsim functions
from vecsim import ket, QReg
import numpy as np

# Also import the gate matrices if you want to use them directly
from vecsim import X, Y, Z, H, CNOT, I, S, CPHASE

## Creating Quantum States

You can create quantum states using the `ket()` function with strings:

In [2]:
# Single qubit states
q0 = ket('0')  # |0⟩ state
q1 = ket('1')  # |1⟩ state
qplus = ket('+')  # (|0⟩+|1⟩)/√2 superposition
qminus = ket('-')  # (|0⟩-|1⟩)/√2 superposition

print("q0:", q0)
print("q1:", q1)
print("qplus:", qplus)
print("qminus:", qminus)

q0: 1.0|0>
q1: 1.0|1>
qplus: 0.7071067811865476|0> 0.7071067811865476|1>
qminus: 0.7071067811865476|0> -0.7071067811865476|1>


In [3]:
# Multi-qubit states
q00 = ket('00')  # |00⟩ two-qubit state
q01 = ket('01')  # |01⟩ 
q10 = ket('10')  # |10⟩
q11 = ket('11')  # |11⟩

print("q00:", q00)
print("q11:", q11)

# You can also create states with superpositions
qpp = ket('++')  # Both qubits in + state
print("q++:", qpp)

q00: 1.0|00>
q11: 1.0|11>
q++: 0.5|00> 0.5|01> 0.5|10> 0.5|11>


## Applying Quantum Gates

Gates are applied in-place and return self for method chaining. Qubit indexing is zero-based from the right.

In [4]:
# X gate (NOT gate) flips |0⟩ ↔ |1⟩
state = ket('0')
print("Before X:", state)
state.X(0)  # Apply X to qubit 0
print("After X:", state)

# Hadamard gate creates superposition
state = ket('0')
print("\nBefore H:", state)
state.H(0)
print("After H:", state)

# Method chaining
result = ket('00').X(0).X(1)
print("\nApply X to both qubits:", result)

Before X: 1.0|0>
After X: 1.0|1>

Before H: 1.0|0>
After H: 0.7071067811865475|0> 0.7071067811865475|1>

Apply X to both qubits: 1.0|11>


## Creating Entangled States (Bell States)

Use CNOT gates to create entanglement between qubits.

In [5]:
# Create a Bell state: (|00⟩ + |11⟩)/√2
bell_state = ket('00').H(0).CNOT(0, 1)
print("Bell state |Φ+⟩:", bell_state)

# The state is now entangled - measuring one qubit determines the other!
# Access the underlying state vector
print("\nState vector:", bell_state.v)
print("Number of qubits:", bell_state.n)

Bell state |Φ+⟩: 0.7071067811865475|00> 0.7071067811865475|11>

State vector: [0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]
Number of qubits: 2


## Exploring the State Vector

Every QReg object has a `.v` attribute containing the NumPy array of complex amplitudes.

In [6]:
# Explore a superposition state
state = ket('+')
print("State:", state)
print("Amplitudes:", state.v)
print("Probabilities:", np.abs(state.v)**2)

# For multi-qubit states
state3 = ket('+++')
print("\nThree-qubit state:")
print("Number of basis states:", len(state3.v))
print("All amplitudes equal:", state3.v)

State: 0.7071067811865476|0> 0.7071067811865476|1>
Amplitudes: [0.70710678+0.j 0.70710678+0.j]
Probabilities: [0.5 0.5]

Three-qubit state:
Number of basis states: 8
All amplitudes equal: [0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j
 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j]


## Quantum Teleportation

Quantum teleportation transfers a quantum state from Alice to Bob using:
1. A shared Bell pair (entanglement)
2. Two classical bits of communication
3. Local operations only

**Protocol:**
- Qubit 0: Alice's state to teleport
- Qubit 1: Alice's half of Bell pair
- Qubit 2: Bob's half of Bell pair

We'll run this on both **vecsim** (state vectors) and **stabilizer** (tableau) simulators.

In [ ]:
from vecsim import ket as v_ket
from stabilizer import ket as s_ket

def teleport_vecsim(initial_state='0'):
    """
    Quantum teleportation using vecsim state vector simulation.
    
    Args:
        initial_state: '0', '1', '+', or '-' for the state to teleport
    
    Returns:
        Tuple of (m1, m2, final_state_string)
    """
    # Create 3-qubit state: |initial⟩ ⊗ |00⟩
    # Then create Bell pair on qubits 1,2
    state = v_ket(initial_state + '00')
    
    # Create Bell pair between qubits 1 and 2
    state.H(1).CNOT(1, 2)
    
    # Alice's Bell measurement: CNOT then H on her qubits
    state.CNOT(0, 1).H(0)
    
    # Alice measures qubits 0 and 1
    m0 = state.M(0)[0]
    m1 = state.M(1)[0]
    
    # Bob applies corrections based on Alice's measurements
    if m1 == 1:
        state.X(2)  # If qubit 1 was |1⟩, apply X
    if m0 == 1:
        state.Z(2)  # If qubit 0 was |1⟩, apply Z
    
    return m0, m1, state

def teleport_stabilizer(initial_state='0'):
    """
    Quantum teleportation using stabilizer tableau simulation.
    
    Args:
        initial_state: '0', '1', '+', or '-' for the state to teleport
    
    Returns:
        Tuple of (m1, m2, stabilizer_generators)
    """
    # Create 3-qubit state: |initial⟩ ⊗ |00⟩
    state = s_ket(initial_state + '00')
    
    # Create Bell pair between qubits 1 and 2
    state.H(1).CNOT(1, 2)
    
    # Alice's Bell measurement: CNOT then H on her qubits
    state.CNOT(0, 1).H(0)
    
    # Alice measures qubits 0 and 1
    m0 = state.M(0)[0]
    m1 = state.M(1)[0]
    
    # Bob applies corrections based on Alice's measurements
    if m1 == 1:
        state.X(2)
    if m0 == 1:
        state.Z(2)
    
    return m0, m1, state

# Test teleportation with the same random seed for both simulators
print("=== Quantum Teleportation Comparison ===\n")

for initial in ['0', '1', '+', '-']:
    print(f"Teleporting |{initial}⟩:")
    
    # Use same seed for fair comparison
    np.random.seed(42)
    v_m0, v_m1, v_state = teleport_vecsim(initial)
    
    np.random.seed(42)
    s_m0, s_m1, s_state = teleport_stabilizer(initial)
    
    print(f"  vecsim:     measurements=({v_m0},{v_m1}), Bob's qubit state from full vector")
    print(f"  stabilizer: measurements=({s_m0},{s_m1}), Bob's qubit generators: {s_state.generators()}")
    
    # Verify Bob's qubit is in the correct state by measuring it
    # For |0⟩ and |1⟩: deterministic result
    # For |+⟩ and |-⟩: Bob's qubit should be in X eigenstate
    np.random.seed(999)
    v_final = v_state.M(2)[0]
    np.random.seed(999)
    s_final = s_state.M(2)[0]
    
    print(f"  Final measurement of Bob's qubit: vecsim={v_final}, stabilizer={s_final}")
    print()

In [ ]:
# Verification: Run many trials and check statistics match
print("=== Statistical Verification (100 trials each) ===\n")

def run_trials(initial_state, n_trials=100):
    """Run teleportation many times and collect Bob's final measurement."""
    v_results = []
    s_results = []
    
    for i in range(n_trials):
        np.random.seed(i)
        _, _, v_state = teleport_vecsim(initial_state)
        v_results.append(v_state.M(2)[0])
        
        np.random.seed(i)
        _, _, s_state = teleport_stabilizer(initial_state)
        s_results.append(s_state.M(2)[0])
    
    return v_results, s_results

for initial in ['0', '1', '+', '-']:
    v_res, s_res = run_trials(initial)
    v_zeros = v_res.count(0)
    s_zeros = s_res.count(0)
    
    # Expected: |0⟩ -> always 0, |1⟩ -> always 1, |+⟩/|-⟩ -> 50/50
    expected = {
        '0': "100% zeros",
        '1': "100% ones",
        '+': "~50% zeros",
        '-': "~50% zeros"
    }
    
    print(f"|{initial}⟩: vecsim={v_zeros}/100 zeros, stabilizer={s_zeros}/100 zeros")
    print(f"     Expected: {expected[initial]}")
    
    # Check correctness
    if initial == '0':
        assert v_zeros == 100 and s_zeros == 100, "Teleport |0⟩ failed!"
    elif initial == '1':
        assert v_zeros == 0 and s_zeros == 0, "Teleport |1⟩ failed!"
    print()

print("Teleportation verified! Both simulators produce equivalent results.")

## Experiment Here

Try your own quantum circuits! Available gates:
- `X(qubit)` - Pauli-X (NOT)
- `Y(qubit)` - Pauli-Y
- `Z(qubit)` - Pauli-Z  
- `H(qubit)` - Hadamard
- `CNOT(control, target)` - Controlled-NOT

In [ ]:
# Your code here - create and manipulate quantum states!

